## Yahoo japan (news) 댓글 클롤링 - 번역

### 선행
1. 야후 일본 뉴스 사이트(https://news.yahoo.co.jp/) 에서 뉴스 링크 복사
2. 아래 코드중 크롬드라이브 경로 수정

## 샘플 기사
- https://news.yahoo.co.jp/articles/71391e90c7ca275f6fc7f9af4475c95d0c5368c2
- https://news.yahoo.co.jp/articles/ae9a5602d46b3c6439d3384bb164c43dc935f7bf  - 이강이 이적 (쿠보)
- https://news.yahoo.co.jp/articles/3f3c8d1d4be77c1611c14ae584b55778cd662066  - 누리호
- https://news.yahoo.co.jp/articles/46f0716aa7dbe8be3b08d9349860b4b0b7f698fc  - 누리호
- https://news.yahoo.co.jp/articles/b155562a3b899e8842fbbc4269f398934c97be6b  - 공군부대 마크 관련

In [6]:
# 그냥 기사사이트 찾기
# 그냥 기사를 누름
import os
import re
from bs4 import BeautifulSoup     
from selenium import webdriver
import time

from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
from datetime import datetime

# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
# driver 실행

print("=" * 80)
url = input(" * 댓글 확인 하려는 기사의 주소를 입력하세요. : ")
f_dir = input(" * 저장할 폴더명을 입력하세요. :")
print(" * 파일명은 기사제목으로 저장 ")

# path = "D:/00_Limhs/py_data/drv/chromedriver.exe"
path = input(" * 이 컴퓨터의 크롬 드라이브 path을 입력하세요.. : ")
driver = webdriver.Chrome(path, options=options)

driver.get(url)

time.sleep(1.5)
element1 = driver.find_element_by_xpath("/html/body/div[1]/main/div[1]/div/article/header/div/div[1]/div/div[1]/div/a")
ActionChains(driver).move_to_element(element1).click(element1).perform()
#댓글 누르기

time.sleep(1.5)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
h1 = soup.find("h1","sc-dyGzUR iNmqKa").text  # 기사 제목부분의 위치로 종종 변경됨. 에러 발생시 확인후 수정
print(h1)
li = soup.find("iframe","news-comment-plguin-iframe")
li = str(li).replace("amp;","").replace(" style=","\n\n\n\n").replace(" src=","\n\n\n\n").replace("_num=2","_num=50").replace("short","full").replace("banner=on","banner=on")

print(li)

# 여기셀을 주소뽑아내는 쎌

url1 = "https://translate.google.com/?hl=ko&sl=ja&tl=ko&text="
url2 = h1
url3 = "&op=translate"
url = url1+url2+url3

driver = webdriver.Chrome(path, options=options)

driver.get(url)
time.sleep(0.6)

# 첫페이지 파씽하기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 


tran1 = soup.find("div","zkZ4Kc dHeVVb").text
tran1 = tran1[:-90]
file = re.compile(r'[^ A-Za-z0-9가-힣+]').sub('',tran1).replace("  ","")
print(file)

re = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=_]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', li)

m = ' '.join(str(s) for s in re)

# numm = input("댓글이 3 -4 -5 선택: ")
numm = 10
# numm = int(numm)
start = time.time()

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

url1 = m
driver = webdriver.Chrome(path)
driver.get(url1)
      
# 파일이 저정될 directory 설정과 폴더 생생. 파일명 지정(생성)
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
ff_dir = f_dir+s
os.makedirs(ff_dir+'/')
os.chdir(ff_dir+'/')

# 첫페이지 파씽하기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')                                        
li= soup.find_all("li","commentListItem")         

df1 = pd.DataFrame(columns=['크롤시간','아이디','몇시간전','긍정', '부정', '댓글'])
df = pd.DataFrame(columns=['크롤시간','아이디','몇시간전','긍정', '부정', '댓글'])

li= soup.find_all("li","commentListItem")
for i in range(len(li)):
    try:
        date = date
        li= soup.find_all("li","commentListItem")[i]    
        id = li.find("a","rapidnofollow").text
        tt = li.find("time","date yjxDate").text.replace("\n","").replace(" ","")
        up1 = li.find("li","good")
        up = up1.find("span","userNum").text
        dn1 = li.find("li","bad")
        dn = dn1.find("span","userNum").text
        com = li.find("p","comment yjxComment").text.replace("\n","")
        df=df.append({'크롤시간':date,'아이디':id,'몇시간전':tt,'긍정':up, '부정':dn, '댓글':com}, ignore_index=True)
    except:
        pass

df1 = pd.concat([df1,df])
f_name = ff_dir+'/'+file
f_name_jp = f_name+"-jp.csv"
f_name_ko = f_name+"-ko.csv"
f_name_ko_jp = f_name+"-kr-jp.csv"

df1.to_csv(f_name_jp,encoding="utf-8-sig",index=True)
#df1.to_csv("C:/123kms/{c}.csv".format(c=file), index=False, mode='w', encoding='utf-8-sig') 
print("첫번째 파싱")

for k in range(1,numm,1):
    start2 = time.time()
    nnn="/html/body/div/section/div/div[2]/div/div/div/ul/div/li[{kk}]/div[1]/article/footer/p/a[3]".format(kk=k)
    element1 = driver.find_element_by_xpath(nnn)
    ActionChains(driver).move_to_element(element1).click(element1).perform()
    
    # 아래로 내리기  - 댓글 펼치기

    time.sleep(0.5)#시간
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.5)#시간
    
    #이것은 아래로 계속누르기
    try:
        for i in range(1,20,1):
            time.sleep(0.3) #시간
            path = "/html/body/div/section/div/div[2]/div/div/div/ul/div/li[{k}]/ul/li[{i}1]/p/a".format(k=k,i=i)
            element1 = driver.find_element_by_xpath(path)
            ActionChains(driver).move_to_element(element1).click(element1).perform()
            time.sleep(0.3)
    except:
        pass
    
    # 댓글 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    df = pd.DataFrame(columns=['크롤시간','아이디','몇시간전','긍정', '부정', '댓글'])

    li= soup.find_all("li","responseItem done")
    
    for i in range(len(li)):
        try:
            date = date
            li= soup.find_all("li","responseItem done")[i]    
            id = li.find("h1","name yjxName").text
            tt = li.find("time","date yjxDate").text.replace("\n","").replace(" ","")
            up1 = li.find("li","good")        
            up = up1.find("span","userNum").text        
            dn1 = li.find("li","bad")
            dn = dn1.find("span","userNum").text
            com = li.find("p","comment yjxComment").text.replace("\n","")
            df=df.append({'크롤시간':date,'아이디':id,'몇시간전':tt,'긍정':up, '부정':dn, '댓글':com}, ignore_index=True)
        except:
            pass

    df1 = pd.concat([df1,df])
    df1 = df1.drop_duplicates(['댓글'], keep='first')
    df1.to_csv(f_name_jp,encoding="utf-8-sig",index=True)
    time.sleep(0.3)
    end = time.time()
    #print(k+1," 번 저장 / 20번 중에","///전체",f"{end - start:.5f} 초 /// 한바퀴",f"{end - start2:.5f} 초")

print("============= 완료 되었습니다. ===================")
end = time.time()

# 파일을 열어서 보기 1 잘열리는지
df2=pd.read_csv(f_name_jp,encoding="utf-8-sig")

#정열을 다시 하기
df2= df2.sort_values(["긍정"], ascending=False)

# 재 저장
df2.to_csv(f_name_jp, index=False, mode='w', encoding='utf-8-sig') 
#한번보기
print(f" {end - start:.5f} sec 시간이 소요되었습니다.")

# 크롤링 완료 - 번역 시작

time.sleep(1)
print("============= 댓글 번역 시작합니다. ===================")
# 1데이터를 한번구경하기
df2 = pd.read_csv(f_name_jp,encoding="utf-8-sig")

df3 = df2['댓글']
df3 = df3[:100]

start = time.time()

# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
# driver 실행
a = len(df3)
print(" 번역 댓글 수 - ",a)

dff = pd.DataFrame(columns=['한글'])
dff1 = pd.DataFrame(columns=['한글'])

print("=" * 80)
print(" " * 30 ,"번역 진행 중", " " * 30)
print("=" * 80)

for i in range(len(df3)):
    start2 = time.time()
    url1 = "https://translate.google.com/?hl=ko&sl=ja&tl=ko&text="
    url2 = df3[i]
    url3 = "&op=translate"
    url = url1 + url2 + url3
    #print(url2)

    path = "D:/00_Limhs/py_data/drv/chromedriver.exe"
    driver = webdriver.Chrome(path, options = options)

    driver.get(url)
    time.sleep(0.6)

    # 첫페이지 파씽하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # print(soup)
    try:
        tran1 = soup.find("div","zkZ4Kc dHeVVb").text
        tran1 = tran1[:-90]
        #print(tran1)
    except:
        tran1 = "ppppp 오류 pppppp"
        print(tran1)
        pass
    
    dff1=dff1.append({'한글':tran1}, ignore_index=True)
#     f_name_ko = f_name+"-ko.csv"
#     f_name_ko_jp = f_name+"-kr-jp.csv"
    dff1.to_csv(f_name_ko,encoding="utf-8-sig",index=True, mode='w')   
    driver.quit()
    end = time.time()
    print(a,"번 중 ", i+1,"번 완료 //","번역에 ", f"{end - start2:.2f} 초 ", "현재까지 총 ", f" {end - start:.2f} 초 소요되었습니다.")

print("댓글 크롤링 : ", f_name_jp)

dff1.to_csv(f_name_ko,encoding="utf-8-sig",index=True, mode='w')

df2 = pd.read_csv(f_name_jp,encoding="utf-8-sig")

df3 = df2[:100]
df3.tail(7)
print("댓글 번역 : ", f_name_ko)
df3.tail(7)

dff=pd.read_csv(f_name_ko,encoding="utf-8-sig")
dff=dff[:100]

print("번역 통합 : ", f_name_ko_jp)
dff.tail(7)

#주의 위에것을 잘 볼것
result = pd.concat([df3,dff],axis=1)

# 의미없은 행 삭제
del result['Unnamed: 0']

# 최종 파일 저장
result.to_csv(f_name_ko_jp,encoding="utf-8-sig",index=True, mode='w') 

# 저장된 파일 열기
exp = os.path.realpath(f_name_ko_jp)
os.startfile(exp)

 * 댓글 확인 하려는 기사의 주소를 입력하세요. : https://news.yahoo.co.jp/articles/fc01c7ffe50ee26166257d2e1612a1ba7cd8d8f1
 * 저장할 폴더명을 입력하세요. :d:\data\
 * 파일명은 기사제목으로 저장 
 * 이 컴퓨터의 크롬 드라이브 path을 입력하세요.. : D:/00_Limhs/py_data/drv/chromedriver.exe
【韓国】初の国産ロケット打ち上げ　衛星の軌道投入には失敗
<iframe class="news-comment-plguin-iframe" frameborder="0" name="news-cmt" scrolling="no"



"https://news.yahoo.co.jp/comment/plugin/v1/full/?origin=https%3A%2F%2Fnews.yahoo.co.jp&topic_id=20211022-00000011-nna_kyodo&space_id=2078709503&content_id=&full_page_url=https%3A%2F%2Fheadlines.yahoo.co.jp%2Fcm%2Fmain%3Fd%3D20211022-00000011-nna_kyodo-kr&comment_num=50&ref=&bkt=&flt=1&grp=&opttype=&disable_total_count=&compact=on&compact_initial_view=&display_author_banner=on&mtestid=mfn_13851%3Dart58t5&display_blurred_comment=on&device=pc&ua=Mozilla%2F5.0%20(Windows%20NT%2010.0%3B%20Win64%3B%20x64)%20AppleWebKit%2F537.36%20(KHTML%2C%20like%20Gecko)%20HeadlessChrome%2F94.0.4606.81%20Safari%2F537.36"



"width: 100%; height: 388px; border: no

# AS코너

### 문제점

1. 사이트가 맞게 들어갔는지
2. 제대로 댓글을 클릭 했는지
3. 파써로 링크를 제대로 따왔는지 -> 아이프레임 에서 링크가 왔는지
4. 리플레이스가 잘 잘라졌는지

### 문제 해결 방법

1. 링크가 제대로 들어갔는지
2. 링크가 안들어가서 파씽이 안됨
3. li 주소가 바뀌었는지
4. id tt-time up dn com 주소가 바뀌었는지 - 바뀌었으면 두개를 바꿔 주어야함
5. 세이브 링크가 잘못된건아닌지
6. nnn링크가 잘되었는지 - 이것은 클릭을 한번 한것이다. 
7. path링크가 잘되었는지 - 이것은 아래로 이동해서 더보기를 클릭한 것이다. 
나머지는 크게 변동될 것이 없다.